In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_json('decisions1')

In [3]:
data = df.iloc[10,1]

In [1]:
len("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") * "*"

'*************************************************************************'

In [ ]:
import requests
import json

API_KEY = '*************************************************************************' # внутри скобок свой апи ключ отсюда https://openrouter.ai/settings/keys
MODEL = "deepseek/deepseek-chat-v3-0324"

def process_content(content):
    return content.replace('<think>', '').replace('</think>', '')

def chat_stream(prompt):
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "stream": True
    }

    with requests.post(
        "https://openrouter.ai/api/v1/chat/completions",
        headers=headers,
        json=data,
        stream=True
    ) as response:
        if response.status_code != 200:
            print("Ошибка API:", response.status_code)
            return ""

        full_response = []
        
        for chunk in response.iter_lines():
            if chunk:
                chunk_str = chunk.decode('utf-8').replace('data: ', '')
                try:
                    chunk_json = json.loads(chunk_str)
                    if "choices" in chunk_json:
                        content = chunk_json["choices"][0]["delta"].get("content", "")
                        if content:
                            cleaned = process_content(content)
                            #print(cleaned, end='', flush=True)
                            full_response.append(cleaned)
                except:
                    pass

        #print()  # Перенос строки после завершения потока
        return ''.join(full_response)

print("Чат с DeepSeek-R1 (by Antric)\nДля выхода введите 'exit'\n")

#user_input = input()

output = []

for i in range(30):
    data = df.iloc[3+7*i,1]
    user_input = "Представь, что ты юрист. Тебе даётся судебное решение со следующим текстом:\
    '{}'. Сформулируй краткий контекст происшествия, \
    три вопроса, которые мог бы задать истец перед обращением в \
    суд и ответы на них, учитывая материалы дела. В ответах, при \
    возможности, ссылайся на статьи закона, указанные в решение дела. \
    Выдай ответ в виде: 'Контекст' : текст контекста, 'Вопрос 1': текст вопроса 1, 'Ответ 1': текст ответа 1 и так далее без лишних знаков\
    В ответах не ссылайся на факты самого дела, а отвечай, \
    как будто истец задал вопрос юристу. Формулируя контекст, не ссылайся на \
    итоговое решение суда, приведи только факты перед заседанием.".format(data)
        
#if user_input.lower() == 'exit':
    #print("Завершение работы...")

            
#print("DeepSeek-R1:", end=' ', flush=True)
    output.append(chat_stream(user_input))

Чат с DeepSeek-R1 (by Antric)
Для выхода введите 'exit'



In [47]:
import re

def split_text_into_blocks(text):
    # Шаблон для поиска ключевых слов вида **Слово**
    pattern = r'(\*\*[А-Яа-яЁёA-Za-z0-9 ]+\*\*\:)'
    blocks = re.split(pattern, text)
    
    # Удаляем пустые строки и лишние пробелы
    blocks = [block.strip() for block in blocks if block.strip()]
    
    # Собираем блоки в словарь
    result = {}
    current_key = None
    
    for block in blocks:
        # Проверяем, является ли блок ключевым словом (начинается и заканчивается **)
        if block.startswith('**') and block.endswith('**:'):
            current_key = block[2:-3]  # убираем ** и приводим к нижнему регистру
        elif current_key:
            result[current_key] = block
    
    return result

In [48]:
def get_df_from_answers(llm_output):
    
    context = []
    answ1 = []
    answ2 = []
    answ3 = []
    quest1 = []
    quest2 = []
    quest3 = []
    for answ in llm_output:
        s = split_text_into_blocks(answ)
        context.append(s['Контекст'])
        answ1.append(s['Ответ 1'])
        answ2.append(s['Ответ 2'])
        answ3.append(s['Ответ 3'])
        quest1.append(s['Вопрос 1'])
        quest2.append(s['Вопрос 2'])
        quest3.append(s['Вопрос 3'])

    dict = {'Контекст': context, 'Вопрос 1': quest1, 'Вопрос 2': \
            quest2, 'Вопрос 3': quest3, 'Ответ 1': answ1,'Ответ 2': answ2, 'Ответ 3': answ3}
    return pd.DataFrame(dict)

In [49]:
get_df_from_answers(output)

,Контекст,Вопрос 1,Вопрос 2,Вопрос 3,Ответ 1,Ответ 2,Ответ 3
0,"Административный истец, председатель совета МК...",Можно ли признать размещенные на фасаде дома к...,Требуется ли согласие собственников МКД для ра...,Какие действия может предпринять УФАС при выяв...,"Да, если информация направлена на привлечение ...","Да, обязательно. Согласно п. 6.3.1.2 Правил ус...","УФАС вправе возбудить дело о нарушении, выдать..."
1,Истец является собственником квартиры в многок...,Можно ли взыскать с управляющей компании ущерб...,Какую компенсацию морального вреда можно требо...,Можно ли взыскать с управляющей компании штраф...,"Да, согласно ч. 1 ст. 1064 ГК РФ, вред, причин...",Компенсация морального вреда возможна по ст. 1...,"Да, по п. 6 ст. 13 Закона «О защите прав потре..."
2,Истец (ФИО1) заключил кредитный договор с АО «...,Могу ли я вернуть деньги за услугу «AUTOSAFE T...,Можно ли взыскать компенсацию морального вреда...,"Какой штраф можно взыскать с компании, если он...","Да, согласно ст. 32 Закона РФ «О защите прав п...","Да, ст. 15 Закона «О защите прав потребителей»...",По п. 6 ст. 13 Закона «О защите прав потребите...
3,Истец (ФИО1) заключил с ИП ФИО3 два договора н...,Могу ли я расторгнуть договор на юридические у...,Какую неустойку я могу взыскать с исполнителя ...,Можно ли взыскать компенсацию морального вреда...,"Да, согласно п. 1 ст. 29 Закона о защите прав ...","По п. 3 ст. 31 ЗоЗПП, за каждый день просрочки...","Да, ст. 15 ЗоЗПП предусматривает право на комп..."
4,Истец заключил договор с ООО «СКИЛБОКС» на онл...,Могу ли я потребовать возврата денег за неоказ...,Могу ли я взыскать проценты за просрочку возвр...,Имею ли я право на компенсацию морального вред...,"Да, согласно п. 1 ст. 782 ГК РФ и ст. 32 Закон...","Да, на основании ст. 395 ГК РФ, если ответчик ...","Да, ст. 15 Закона о защите прав потребителей п..."
5,Истцы заключили договор участия в долевом стро...,Могу ли я требовать уменьшения цены договора и...,Положена ли мне неустойка за просрочку устране...,Можно ли взыскать штраф с застройщика за отказ...,"Да, согласно ч. 1 ст. 7 Федерального закона № ...","Да, по ч. 8 ст. 7 214-ФЗ и п. 1 ст. 23 Закона ...",Штраф в размере 50% от присужденной суммы взыс...
6,Истец заключил с застройщиком договор участия ...,Могу ли я потребовать от застройщика возместит...,Как рассчитывается неустойка за просрочку возм...,Могу ли я взыскать компенсацию морального вред...,"Да, согласно ч. 3 ст. 7 Федерального закона №2...",Согласно ч. 8 ст. 7 Федерального закона №214-Ф...,"Да, в соответствии со ст. 15 Закона «О защите ..."
7,Гражданин (истец) передал денежные средства ст...,С какого момента можно требовать проценты за п...,Могу ли я взыскать штраф в размере 50% от сумм...,Будет ли учитываться период моратория на банкр...,"Проценты по ст. 395 ГК РФ начисляются со дня, ...","Да, согласно п. 6 ст. 13 Закона о защите прав ...","Да, в период действия моратория (с 1 апреля по..."
8,Истцы (Косолапов А.И. и Косолапова Ю.В.) заклю...,Можем ли мы потребовать уменьшения цены догово...,"Положена ли нам компенсация морального вреда, ...",Можно ли взыскать штраф с застройщика за отказ...,"Да, согласно ч. 2 ст. 7 Федерального закона № ...","Да, компенсация морального вреда предусмотрена...",Штраф по п. 6 ст. 13 Закона «О защите прав пот...
9,Истец (участник долевого строительства) заключ...,Могу ли я потребовать от застройщика возместит...,Какой размер компенсации морального вреда я мо...,Обязан ли застройщик выплатить штраф за отказ ...,"Да, согласно ч. 2 ст. 7 Федерального закона № ...",Размер компенсации определяется судом индивиду...,"Да, по п. 6 ст. 13 Закона «О защите прав потре..."
